<a href="https://colab.research.google.com/github/rahulbaburaj1/RNN_LSTM_Google_Stocks/blob/main/rnn_lstm_google_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
df = pd.read_csv("GOOG.csv")

In [ ]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-08-08,119.120003,120.860001,117.830002,118.139999,118.139999,17061100
1,2022-08-09,117.989998,118.199997,116.559998,117.500000,117.500000,15424300
2,2022-08-10,119.589996,121.779999,119.360001,120.650002,120.650002,20497000
3,2022-08-11,122.080002,122.339996,119.550003,119.820000,119.820000,16671600
4,2022-08-12,121.160004,122.650002,120.400002,122.650002,122.650002,16121100


In [ ]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       250 non-null    object 
 1   Open       250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Close      250 non-null    float64
 5   Adj Close  250 non-null    float64
 6   Volume     250 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df.set_index(df['Date'],inplace=True)
df.drop(columns=['Open','High','Low','Volume'],inplace=True)

In [ ]:
df.head()

,Date,Close,Adj Close
Date,,,
2022-08-08,2022-08-08,118.139999,118.139999
2022-08-09,2022-08-09,117.500000,117.500000
2022-08-10,2022-08-10,120.650002,120.650002
2022-08-11,2022-08-11,119.820000,119.820000
2022-08-12,2022-08-12,122.650002,122.650002


In [ ]:
import plotly.graph_objects as go


In [ ]:
trace1 = go.Scatter(
    x = df['Date'],
    y = df['Close'],
    mode = 'lines',
    name = 'Data'
)

layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title':"Date"},
    yaxis = {"title":"Close"}

)
fig = go.Figure(data = [trace1],layout = layout)
fig.show()

In [ ]:
close_data = df['Close'].values

In [ ]:
close_data

array([118.139999, 117.5     , 120.650002, 119.82    , 122.650002,
       122.879997, 122.510002, 120.32    , 120.860001, 118.120003,
       115.07    , 114.769997, 114.699997, 117.699997, 111.300003,
       110.339996, 109.910004, 109.150002, 110.550003, 108.68    ,
       107.480003, 110.480003, 109.419998, 111.779999, 111.870003,
       105.309998, 105.870003, 103.900002, 103.629997, 103.849998,
       101.830002, 100.010002, 100.57    ,  99.169998,  98.809998,
        98.089996, 100.739998,  98.089996,  96.150002,  99.300003,
       102.410004, 102.220001, 102.239998,  99.57    ,  98.709999,
        98.050003,  98.300003,  99.709999,  97.18    , 100.779999,
       101.389999, 100.290001, 100.529999, 101.480003, 102.970001,
       104.93    ,  94.82    ,  92.599998,  96.580002,  94.660004,
        90.5     ,  87.07    ,  83.489998,  86.699997,  88.650002,
        88.910004,  87.400002,  94.169998,  96.730003,  96.029999,
        98.720001,  98.989998,  98.5     ,  97.800003,  95.830

In [ ]:
close_data = close_data.reshape(-1,1)

In [ ]:
close_data

array([[118.139999],
       [117.5     ],
       [120.650002],
       [119.82    ],
       [122.650002],
       [122.879997],
       [122.510002],
       [120.32    ],
       [120.860001],
       [118.120003],
       [115.07    ],
       [114.769997],
       [114.699997],
       [117.699997],
       [111.300003],
       [110.339996],
       [109.910004],
       [109.150002],
       [110.550003],
       [108.68    ],
       [107.480003],
       [110.480003],
       [109.419998],
       [111.779999],
       [111.870003],
       [105.309998],
       [105.870003],
       [103.900002],
       [103.629997],
       [103.849998],
       [101.830002],
       [100.010002],
       [100.57    ],
       [ 99.169998],
       [ 98.809998],
       [ 98.089996],
       [100.739998],
       [ 98.089996],
       [ 96.150002],
       [ 99.300003],
       [102.410004],
       [102.220001],
       [102.239998],
       [ 99.57    ],
       [ 98.709999],
       [ 98.050003],
       [ 98.300003],
       [ 99.7

In [ ]:
split_percentage = .80

split =int( len(close_data)*split_percentage)

In [ ]:
split

200

In [ ]:
close_train = close_data[:split]
close_test = close_data[split:]

In [ ]:
date_train = df['Date'][:split]
date_test = df['Date'][split:]

In [ ]:
print(len(close_test))

50


In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
look_back = 15
train_generator =  TimeseriesGenerator(close_train,close_train,length = look_back,batch_size=20)
test_generator = TimeseriesGenerator(close_test,close_test,length = look_back,batch_size=1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense


In [ ]:
model = Sequential()
model.add(
    LSTM(10,activation = 'relu',input_shape = ( look_back,1))
)
model.add(
    Dense(1)
)

In [ ]:
model.compile(optimizer = 'adam',loss = 'mse')

In [ ]:
model.fit_generator(train_generator,epochs=100,verbose=1)#generator used for timeseries

Epoch 1/100
 1/10 [==>...........................] - ETA: 0s - loss: 8.0317

<ipython-input-75-afb9244e47a8>:1: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



10/10 [==============================] - 1s 61ms/step - loss: 24.1795
Epoch 2/100
10/10 [==============================] - 0s 45ms/step - loss: 13.2436
Epoch 3/100
10/10 [==============================] - 0s 29ms/step - loss: 15.4260
Epoch 4/100
10/10 [==============================] - 0s 25ms/step - loss: 16.6414
Epoch 5/100
10/10 [==============================] - 0s 32ms/step - loss: 14.2089
Epoch 6/100
10/10 [==============================] - 0s 26ms/step - loss: 23.2924
Epoch 7/100
10/10 [==============================] - 0s 25ms/step - loss: 17.8823
Epoch 8/100
10/10 [==============================] - 0s 31ms/step - loss: 13.7965
Epoch 9/100
10/10 [==============================] - 0s 28ms/step - loss: 13.9108
Epoch 10/100
10/10 [==============================] - 0s 27ms/step - loss: 15.3642
Epoch 11/100
10/10 [==============================] - 0s 27ms/step - loss: 14.6203
Epoch 12/100
10/10 [==============================] - 0s 30ms/step - loss: 10.8367
Epoch 13/100
10/10 [=====

In [ ]:
prediction = model.predict_generator(test_generator)

<ipython-input-76-342c80f366ab>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
model.save('google.h5')

In [ ]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


In [ ]:
prediction

array([118.60065 , 119.98183 , 118.90583 , 117.64488 , 117.03893 ,
       119.49722 , 120.96981 , 119.686035, 118.57141 , 116.69309 ,
       115.97323 , 116.57806 , 117.54185 , 118.939415, 119.15787 ,
       119.354774, 117.506676, 117.71838 , 118.79921 , 122.399124,
       122.487465, 120.329124, 119.97565 , 120.17537 , 118.88199 ,
       119.83792 , 121.336815, 122.43343 , 123.63335 , 119.58879 ,
       115.720375, 117.88194 , 122.35749 , 126.022446, 127.29663 ],
      dtype=float32)

In [ ]:
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'data'
)
trace2 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'test'
)
trace3 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title':"Date"},
    yaxis = {"title":"Close"}

)
fig = go.Figure(data = ([trace1,trace2,trace3]),layout = layout)
fig.show()